In [1]:
from flask import Flask, render_template, request, redirect, url_for
import random
from faker import Faker
from datetime import datetime
from jinja2 import DictLoader

app = Flask(__name__)
fake = Faker()

# Generate fake data
properties = [{
    "id": i,
    "address": fake.street_address(),
    "city": fake.city(),
    "roof_type": random.choice(["Tile", "Shingle", "Metal"]),
    "last_roof_date": fake.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d'),
    "owner": fake.name(),
    "parcel_name": fake.company(),
    "llc_mailing_address": fake.address(),
    "contact_info": [],
    "notes": []
} for i in range(1, 51)]

# HTML Templates in DictLoader
app.jinja_loader = DictLoader({
    'index.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Roof Permits</title>
    </head>
    <body>
        <h1>Roof Permits</h1>
        <table>
            <tr><th>Address</th><th>City</th><th>Roof Type</th><th>Last Roof Date</th><th>Owner</th></tr>
            {% for prop in properties %}
            <tr onclick="location.href='{{ url_for('edit_property', prop_id=prop.id) }}'">
                <td>{{ prop.address }}</td>
                <td>{{ prop.city }}</td>
                <td>{{ prop.roof_type }}</td>
                <td>{{ prop.last_roof_date }}</td>
                <td>{{ prop.owner }}</td>
            </tr>
            {% endfor %}
        </table>
    </body>
    </html>
    ''',
    'edit_property.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Edit Property</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <script>
            function addContact(){
                document.getElementById('contacts').insertAdjacentHTML('beforeend', '<div class="input-group mb-2"><input type="email" name="email" placeholder="Email" class="form-control"><input type="text" name="phone" placeholder="Phone" class="form-control"><input type="text" name="job_title" placeholder="Job Title" class="form-control"></div>');
            }
        </script>
    </head>
    <body>
        <div class="container py-4">
            <h2>Edit Property Details</h2>
            <form method="post">
                <label class="form-label">Address</label>
                <input class="form-control mb-2" name="address" value="{{ prop.address }}">

                <label class="form-label">Roof Type</label>
                <input class="form-control mb-2" name="roof_type" value="{{ prop.roof_type }}">

                <label class="form-label">Last Roof Date</label>
                <input class="form-control mb-2" type="date" name="last_roof_date" value="{{ prop.last_roof_date }}">

                <label class="form-label">Owner</label>
                <input class="form-control mb-2" name="owner" value="{{ prop.owner }}">

                <label class="form-label">Parcel Name</label>
                <input class="form-control mb-2" name="parcel_name" value="{{ prop.parcel_name }}">

                <label class="form-label">LLC Mailing Address</label>
                <input class="form-control mb-3" name="llc_mailing_address" value="{{ prop.llc_mailing_address }}">

                <label class="form-label">Add Note</label>
                <textarea class="form-control mb-2" name="notes"></textarea>
                <button class="btn btn-secondary mb-3" type="submit">Add Note</button>

                {% for note in prop.notes %}
                    <div class="alert alert-secondary">{{ note }}</div>
                {% endfor %}

                <label class="form-label">Contact Info</label>
                <div id="contacts">
                    {% for c in prop.contact_info %}
                        <div class="input-group mb-2">
                            <input class="form-control" name="email" value="{{ c.email }}">
                            <input class="form-control" name="phone" value="{{ c.phone }}">
                            <input class="form-control" name="job_title" value="{{ c.job_title }}">
                        </div>
                    {% endfor %}
                </div>
                <button type="button" class="btn btn-info mb-3" onclick="addContact()">Add Contact</button><br>
                <button class="btn btn-success">Save Changes</button>
            </form>
        </div>
    </body>
    </html>
    '''
})

# Routes and logic
@app.route('/')
def index():
    address = request.args.get('address', '').lower()
    roof_type = request.args.get('roof_type', '').lower()
    owner = request.args.get('owner', '').lower()
    date_filter = request.args.get('date_filter', '')
    date_from = request.args.get('date_from', '')
    date_to = request.args.get('date_to', '')

    filtered_properties = properties

    if address:
        filtered_properties = [prop for prop in filtered_properties if address in prop['address'].lower() or address in prop['city'].lower()]
    if roof_type:
        filtered_properties = [prop for prop in filtered_properties if roof_type in prop['roof_type'].lower()]
    if owner:
        filtered_properties = [prop for prop in filtered_properties if owner in prop['owner'].lower()]

    if date_filter and date_from:
        date_from_obj = datetime.strptime(date_from, '%Y-%m-%d')
        if date_filter == 'before':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') < date_from_obj]
        elif date_filter == 'after':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') > date_from_obj]
        elif date_filter == 'between' and date_to:
            date_to_obj = datetime.strptime(date_to, '%Y-%m-%d')
            filtered_properties = [prop for prop in filtered_properties if date_from_obj <= datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') <= date_to_obj]

    return render_template('index.html', properties=filtered_properties)

@app.route('/property/<int:prop_id>', methods=['GET', 'POST'])
def edit_property(prop_id):
    prop = next((p for p in properties if p['id'] == prop_id), None)
    if request.method == 'POST':
        prop['address'] = request.form['address']
        prop['roof_type'] = request.form['roof_type']
        prop['last_roof_date'] = request.form['last_roof_date']
        prop['owner'] = request.form['owner']

        note = request.form.get('notes', '').strip()
        if note:
            prop['notes'].append(note)

        emails = request.form.getlist('email')
        phones = request.form.getlist('phone')
        job_titles = request.form.getlist('job_title')

        prop['contact_info'] = [
            {"email": e, "phone": p, "job_title": j}
            for e, p, j in zip(emails, phones, job_titles)
            if e or p or j
        ]

        return redirect(url_for('index'))

    return render_template('edit_property.html', prop=prop)


if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Aug/2025 12:14:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2025 12:14:29] "GET / HTTP/1.1" 200 -
